In [1]:
library(ArchR)
library(Seurat)
library(Signac)
library(magrittr)
library(genomation)
library(GenomicRanges)
library(Matrix)
library(ggplot2)


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .______      
          /   \     |   _ 

Specify file path

In [2]:
path.seurat = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/4.merge_peaks_from_22_cell_types.240523/obj.seurat.rds"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/4.merge_peaks_from_22_cell_types.240523/pairs.rds"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/ArchR/BMMC22.merge.240528/BMMC22.merge.240528/"

In [3]:
n.cores = 16
maxDist = 1000000

In [4]:
addArchRThreads(threads = n.cores) 
addArchRGenome("hg38")

Setting default number of Parallel threads to 16.

Setting default genome to Hg38.



Import seurat object

In [5]:
obj.seurat = readRDS(path.seurat)

Import candidate E-G pairs

In [6]:
pairs.E2G = readRDS(path.pairs.E2G)

Create an ArchRProject

In [7]:
str(Fragments(obj.seurat))

List of 13
 $ :Formal class 'Fragment' [package "Signac"] with 3 slots
  .. ..@ path : chr "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellra"| __truncated__
  .. ..@ hash : chr [1:2] "86350e844e0a3a1991e9c650ccac7f43" "555000121b665f095f13712669597ac8"
  .. ..@ cells: Named chr [1:6224] "TAGTTGTCACCCTCAC-1" "CTATGGCCATAACGGG-1" "CCGCACACAGGTTAAA-1" "TCATTTGGTAATGGAA-1" ...
  .. .. ..- attr(*, "names")= chr [1:6224] "s1d1_TAGTTGTCACCCTCAC-1" "s1d1_CTATGGCCATAACGGG-1" "s1d1_CCGCACACAGGTTAAA-1" "s1d1_TCATTTGGTAATGGAA-1" ...
 $ :Formal class 'Fragment' [package "Signac"] with 3 slots
  .. ..@ path : chr "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellra"| __truncated__
  .. ..@ hash : chr [1:2] "3ee058ae058031e3f233c68b0db9e6b9" "1039e761a9da92608ebabb952d11b3c8"
  .. ..@ cells: Named chr [1:6740] "CTACCGGCAGTTGCGT-1" "GATGCAGCAGCAACAG-1" "TGAGTGAAGAGGAAGG-1" "CCGCTA

In [8]:
ArrowFiles <- createArrowFiles(
  inputFiles = c(Fragments(obj.seurat)[[1]]@path,
                 Fragments(obj.seurat)[[2]]@path,
                 Fragments(obj.seurat)[[3]]@path,
                 Fragments(obj.seurat)[[4]]@path,
                 Fragments(obj.seurat)[[5]]@path,
                 Fragments(obj.seurat)[[6]]@path,
                 Fragments(obj.seurat)[[7]]@path,
                 Fragments(obj.seurat)[[8]]@path,
                 Fragments(obj.seurat)[[9]]@path,
                 Fragments(obj.seurat)[[10]]@path,
                 Fragments(obj.seurat)[[11]]@path,
                 Fragments(obj.seurat)[[12]]@path,
                 Fragments(obj.seurat)[[13]]@path),
  sampleNames = c("s1d1",
                  "s1d2",
                  "s1d3",
                  "s2d1",
                  "s2d4",
                  "s2d5",
                  "s3d10",
                  "s3d3",
                  "s3d6",
                  "s3d7",
                  "s4d1",
                  "s4d8",
                  "s4d9"),
  validBarcodes = list(s1d1 = unname(Fragments(obj.seurat)[[1]]@cells),
                       s1d2 = unname(Fragments(obj.seurat)[[2]]@cells),
                       s1d3 = unname(Fragments(obj.seurat)[[3]]@cells),
                       s2d1 = unname(Fragments(obj.seurat)[[4]]@cells),
                       s2d4 = unname(Fragments(obj.seurat)[[5]]@cells),
                       s2d5 = unname(Fragments(obj.seurat)[[6]]@cells),
                       s3d10 = unname(Fragments(obj.seurat)[[7]]@cells),
                       s3d3 = unname(Fragments(obj.seurat)[[8]]@cells),
                       s3d6 = unname(Fragments(obj.seurat)[[9]]@cells),
                       s3d7 = unname(Fragments(obj.seurat)[[10]]@cells),
                       s4d1 = unname(Fragments(obj.seurat)[[11]]@cells),
                       s4d8 = unname(Fragments(obj.seurat)[[12]]@cells),
                       s4d9 = unname(Fragments(obj.seurat)[[13]]@cells)),
  addTileMat = T,
  addGeneScoreMat = F,
  force = T
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

ArchR logging to : ArchRLogs/ArchR-createArrows-1d7e0c65e11fa5-Date-2024-05-28_Time-19-24-06.052714.log
If there is an issue, please report to github with logFile!

Cleaning Temporary Files

2024-05-28 19:24:06.354269 : Batch Execution w/ safelapply!, 0 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-createArrows-1d7e0c65e11fa5-Date-2024-05-28_Time-19-24-06.052714.log



In [9]:
ArrowFiles

[1] "s3d10.arrow" "s3d3.arrow"  "s3d7.arrow"  "s4d1.arrow"  "s1d1.arrow" 
 [6] "s4d8.arrow"  "s2d4.arrow"  "s2d1.arrow"  "s2d5.arrow"  "s1d2.arrow" 
[11] "s1d3.arrow"  "s4d9.arrow"  "s3d6.arrow"

In [10]:
dir.create(paste(dir.output,"ArchR",sep="/"),recursive = T)

In [11]:
obj.ArchR <- ArchRProject(
  ArrowFiles = ArrowFiles, 
  outputDirectory = paste(dir.output,"ArchR",sep="/"),
  copyArrows = TRUE 
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

Validating Arrows...

Getting SampleNames...



Copying ArrowFiles to Ouptut Directory! If you want to save disk space set copyArrows = FALSE

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 


Getting Cell Metadata...



Merging Cell Metadata...

Initializing ArchRProject...


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__           

In [12]:
obj.ArchR


           ___      .______        ______  __    __  .______      
          /   \     |   _  \      /      ||  |  |  | |   _  \     
         /  ^  \    |  |_)  |    |  ,----'|  |__|  | |  |_)  |    
        /  /_\  \   |      /     |  |     |   __   | |      /     
       /  _____  \  |  |\  \\___ |  `----.|  |  |  | |  |\  \\___.
      /__/     \__\ | _| `._____| \______||__|  |__| | _| `._____|
    



class: ArchRProject 
outputDirectory: /maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/2.Genome_wide_prediction/ArchR/BMMC22.merge.240528/BMMC22.merge.240528/ArchR 
samples(13): s3d10 s3d3 ... s4d9 s3d6
sampleColData names(1): ArrowFiles
cellColData names(13): Sample TSSEnrichment ... nDiFrags BlacklistRatio
numberOfCells(1): 67013
medianTSS(1): 16.107
medianFrags(1): 9747

In [13]:
obj.ArchR = addIterativeLSI(
    ArchRProj = obj.ArchR,
    useMatrix = "TileMatrix", 
    name = "IterativeLSI"
)

Checking Inputs...

ArchR logging to : ArchRLogs/ArchR-addIterativeLSI-1d7e0c2d360b45-Date-2024-05-28_Time-20-08-18.932113.log
If there is an issue, please report to github with logFile!

2024-05-28 20:08:25.886429 : Computing Total Across All Features, 0.105 mins elapsed.

2024-05-28 20:08:33.056514 : Computing Top Features, 0.224 mins elapsed.

###########
2024-05-28 20:08:36.355428 : Running LSI (1 of 2) on Top Features, 0.279 mins elapsed.
###########

2024-05-28 20:08:36.417976 : Sampling Cells (N = 10007) for Estimated LSI, 0.28 mins elapsed.

2024-05-28 20:08:36.421448 : Creating Sampled Partial Matrix, 0.28 mins elapsed.

2024-05-28 20:08:51.729617 : Computing Estimated LSI (projectAll = FALSE), 0.536 mins elapsed.

2024-05-28 20:09:43.334449 : Identifying Clusters, 1.396 mins elapsed.

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
2024-05-28 20:09:55.453801 : Identified 6 Clusters, 1.598 mins elapsed.

2024-05-28 20:09:55.473286 : Saving LSI Iteration, 1.5


************************************************************
2024-05-28 20:10:13.337233 : ERROR Found in .saveIteration for  
LogFile = ArchRLogs/ArchR-addIterativeLSI-1d7e0c2d360b45-Date-2024-05-28_Time-20-08-18.932113.log

<simpleError in g$grobs[[legend]]: no such index at level 2
>

************************************************************



2024-05-28 20:10:13.341476 : Creating Cluster Matrix on the total Group Features, 1.896 mins elapsed.

2024-05-28 20:10:42.249095 : Computing Variable Features, 2.378 mins elapsed.

###########
2024-05-28 20:10:42.426643 : Running LSI (2 of 2) on Variable Features, 2.381 mins elapsed.
###########

2024-05-28 20:10:42.456429 : Creating Partial Matrix, 2.381 mins elapsed.

2024-05-28 20:11:11.055925 : Computing LSI, 2.858 mins elapsed.

2024-05-28 20:12:51.905619 : Finished Running IterativeLSI, 4.539 mins elapsed.



In [14]:
obj.ArchR = addPeakSet(obj.ArchR,
                       peakSet = reduce(pairs.E2G))

In [15]:
obj.ArchR = addPeakMatrix(obj.ArchR)

ArchR logging to : ArchRLogs/ArchR-addPeakMatrix-1d7e0c69cf2d73-Date-2024-05-28_Time-20-12-57.652409.log
If there is an issue, please report to github with logFile!

2024-05-28 20:12:58.040229 : Batch Execution w/ safelapply!, 0 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-addPeakMatrix-1d7e0c69cf2d73-Date-2024-05-28_Time-20-12-57.652409.log



In [16]:
seRNA = import10xFeatureMatrix(c("/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site1_donor1/outs/filtered_feature_bc_matrix.h5",
                                 "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site1_donor2/outs/filtered_feature_bc_matrix.h5",
                                 "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site1_donor3/outs/filtered_feature_bc_matrix.h5",
                                 "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site2_donor1/outs/filtered_feature_bc_matrix.h5",
                                 "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site2_donor4/outs/filtered_feature_bc_matrix.h5",
                                 "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site2_donor5/outs/filtered_feature_bc_matrix.h5",
                                 "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site3_donor10/outs/filtered_feature_bc_matrix.h5",
                                 "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site3_donor3/outs/filtered_feature_bc_matrix.h5",
                                 "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site3_donor6/outs/filtered_feature_bc_matrix.h5",
                                 "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site3_donor7/outs/filtered_feature_bc_matrix.h5",
                                 "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site4_donor1/outs/filtered_feature_bc_matrix.h5",
                                 "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site4_donor8/outs/filtered_feature_bc_matrix.h5",
                                 "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site4_donor9/outs/filtered_feature_bc_matrix.h5"),
                               c("s1d1",
                                 "s1d2",
                                 "s1d3",
                                 "s2d1",
                                 "s2d4",
                                 "s2d5",
                                 "s3d10",
                                 "s3d3",
                                 "s3d6",
                                 "s3d7",
                                 "s4d1",
                                 "s4d8",
                                 "s4d9"))

Importing Feature Matrix 1 of 13

Importing Feature Matrix 2 of 13

Importing Feature Matrix 3 of 13

Importing Feature Matrix 4 of 13

Importing Feature Matrix 5 of 13

Importing Feature Matrix 6 of 13

Importing Feature Matrix 7 of 13

Importing Feature Matrix 8 of 13

Importing Feature Matrix 9 of 13

Importing Feature Matrix 10 of 13

Importing Feature Matrix 11 of 13

Importing Feature Matrix 12 of 13

Importing Feature Matrix 13 of 13

Re-ordering RNA matricies for consistency.

Merging individual RNA objects...


Merging s1d2



ERROR: Error in import10xFeatureMatrix(c("/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site1_donor1/outs/filtered_feature_bc_matrix.h5", : Error - rownames (genes) of individual RNA objects are not equivalent.


In [38]:
paths.h5 = c("/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site1_donor1/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site1_donor2/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site1_donor3/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site2_donor1/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site2_donor4/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site2_donor5/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site3_donor10/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site3_donor3/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site3_donor6/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site3_donor7/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site4_donor1/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site4_donor8/outs/raw_feature_bc_matrix.h5",
             "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site4_donor9/outs/raw_feature_bc_matrix.h5")
names.sample = c("s1d1",
                 "s1d2",
                 "s1d3",
                 "s2d1",
                 "s2d4",
                 "s2d5",
                 "s3d10",
                 "s3d3",
                 "s3d6",
                 "s3d7",
                 "s4d1",
                 "s4d8",
                 "s4d9")
list.seRNA = lapply(1:length(names.sample),function(x){
    import10xFeatureMatrix(paths.h5[x],
                           names.sample[x])
})

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.

Importing Feature Matrix 1 of 1

Re-ordering RNA matricies for consistency.


In [39]:
list.seRNA

[[1]]
class: RangedSummarizedExperiment 
dim: 61218 727198 
metadata(0):
assays(1): counts
rownames(61218): DDX11L1 MIR6859-1 ... RNA5-8SN5 ENSG00000273937
rowData names(5): feature_type genome id interval name
colnames(727198): s1d1#AAACAGCCAAACAACA-1 s1d1#AAACAGCCAAACATAG-1 ...
  s1d1#TTTGTTGGTTTGTCTA-1 s1d1#TTTGTTGGTTTGTGGA-1
colData names(0):

[[2]]
class: RangedSummarizedExperiment 
dim: 61218 726937 
metadata(0):
assays(1): counts
rownames(61218): DDX11L2 DDX11L1 ... RNA5-8SN5 ENSG00000273937
rowData names(5): feature_type genome id interval name
colnames(726937): s1d2#AAACAGCCAAACAACA-1 s1d2#AAACAGCCAAACATAG-1 ...
  s1d2#TTTGTTGGTTTGTGGA-1 s1d2#TTTGTTGGTTTGTTGC-1
colData names(0):

[[3]]
class: RangedSummarizedExperiment 
dim: 61218 724480 
metadata(0):
assays(1): counts
rownames(61218): DDX11L1 MIR6859-1 ... RNA5-8SN5 ENSG00000273937
rowData names(5): feature_type genome id interval name
colnames(724480): s1d3#AAACAGCCAAACAACA-1 s1d3#AAACAGCCAAACATAG-1 ...
  s1d3#TTTGTTGGTTTGTG

In [40]:
id.intersect = rowData(list.seRNA[[1]])[,"id"]
length(id.intersect)
for(n.tmp in 2:13){
    id.intersect = intersect(id.intersect,rowData(list.seRNA[[n.tmp]])[,"id"])
}
length(id.intersect)

[1] 61218

[1] 61165

In [41]:
seRNA = list.seRNA[[1]][rowData(list.seRNA[[1]])[,"id"] %in% id.intersect]
for(n.tmp in 2:13){
    seRNA = cbind(seRNA,list.seRNA[[n.tmp]][rowData(list.seRNA[[n.tmp]])[,"id"] %in% id.intersect])
}
seRNA

class: RangedSummarizedExperiment 
dim: 61165 9222766 
metadata(0):
assays(1): counts
rownames(61165): DDX11L1 MIR6859-1 ... RNA5-8SN5 ENSG00000273937
rowData names(5): feature_type genome id interval name
colnames(9222766): s1d1#AAACAGCCAAACAACA-1 s1d1#AAACAGCCAAACATAG-1 ...
  s4d9#TTTGTTGGTTTGTGGA-1 s4d9#TTTGTTGGTTTGTTGC-1
colData names(0):

In [42]:
seRNA = seRNA[,rownames(obj.ArchR@cellColData)]

In [43]:
seRNA

class: RangedSummarizedExperiment 
dim: 61165 67013 
metadata(0):
assays(1): counts
rownames(61165): DDX11L1 MIR6859-1 ... RNA5-8SN5 ENSG00000273937
rowData names(5): feature_type genome id interval name
colnames(67013): s3d10#GTAGCTGTCAGCAAAG-1 s3d10#AACTAGTGTTACCTGT-1 ...
  s3d6#TCACCGGCAGCCTTGG-1 s3d6#GAACCTGTCGTTACAA-1
colData names(0):

In [44]:
obj.ArchR = addGeneExpressionMatrix(obj.ArchR,
                                    seRNA = seRNA,
                                    strictMatch = T)

ArchR logging to : ArchRLogs/ArchR-addGeneExpressionMatrix-1d7e0c6cadaeb2-Date-2024-05-31_Time-14-42-52.31355.log
If there is an issue, please report to github with logFile!

Overlap w/ scATAC = 1

2024-05-31 14:42:54.964458 : 

Overlap Per Sample w/ scATAC : s1d1=6149,s1d2=6685,s1d3=4260,s2d1=3966,s2d4=5321,s2d5=4558,s3d10=6724,s3d3=4269,s3d6=1645,s3d7=1627,s4d1=7905,s4d8=9705,s4d9=4199

2024-05-31 14:42:55.158768 : 

2024-05-31 14:43:02.083834 : Batch Execution w/ safelapply!, 0 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-addGeneExpressionMatrix-1d7e0c6cadaeb2-Date-2024-05-31_Time-14-42-52.31355.log



Run ArchR prediction

In [45]:
start_time <- Sys.time()
obj.ArchR = addPeak2GeneLinks(
    ArchRProj = obj.ArchR,
    reducedDims = "IterativeLSI",
    useMatrix = "GeneExpressionMatrix",
    maxDist = maxDist,
    threads = n.cores
)
end_time <- Sys.time()
execution_time <- end_time - start_time

ArchR logging to : ArchRLogs/ArchR-addPeak2GeneLinks-1d7e0c3d642585-Date-2024-05-31_Time-14-47-22.429103.log
If there is an issue, please report to github with logFile!

2024-05-31 14:47:22.807543 : Getting Available Matrices, 0.006 mins elapsed.

2024-05-31 14:47:48.406844 : Filtered Low Prediction Score Cells (0 of 67013, 0), 0.175 mins elapsed.

2024-05-31 14:47:58.560016 : Computing KNN, 0.344 mins elapsed.

2024-05-31 14:47:58.986131 : Identifying Non-Overlapping KNN pairs, 0.351 mins elapsed.

2024-05-31 14:48:00.63085 : Identified 500 Groupings!, 0.378 mins elapsed.

2024-05-31 14:48:00.69663 : Getting Group RNA Matrix, 0.38 mins elapsed.

2024-05-31 14:48:43.11274 : Getting Group ATAC Matrix, 1.087 mins elapsed.

2024-05-31 14:50:59.200066 : Normalizing Group Matrices, 3.355 mins elapsed.

2024-05-31 14:51:18.672297 : Finding Peak Gene Pairings, 3.679 mins elapsed.

2024-05-31 14:51:25.100427 : Computing Correlations, 3.786 mins elapsed.

2024-05-31 14:53:00.81526 : Completed P

In [46]:
execution_time

Time difference of 5.640439 mins

In [47]:
p2g = metadata(obj.ArchR@peakSet)$Peak2GeneLinks

In [48]:
metadata(p2g)[[1]]$PeakName = paste(seqnames(metadata(p2g)[[1]]),
                                    ranges(metadata(p2g)[[1]]),sep="-")

In [49]:
p2g$PairName = paste(metadata(p2g)[[1]][p2g$idxATAC]$PeakName,
                     metadata(p2g)[[2]][p2g$idxRNA]$name,
                     sep = "_")

In [50]:
rownames(p2g) = p2g$PairName
p2g

DataFrame with 30762630 rows and 7 columns
                                  idxATAC    idxRNA Correlation       FDR
                                <integer> <integer>   <numeric> <numeric>
chr1-10010-10536_DDX11L1                1         1         NaN       NaN
chr1-96496-96699_DDX11L1                2         1         NaN       NaN
chr1-115495-115930_DDX11L1              3         1         NaN       NaN
chr1-180470-181899_DDX11L1              4         1         NaN       NaN
chr1-184379-184607_DDX11L1              5         1         NaN       NaN
...                                   ...       ...         ...       ...
chrX-156008869-156009386_WASH6P    540425     60591  0.09305258  0.174782
chrX-156019809-156020162_WASH6P    540426     60591 -0.03901933  0.727297
chrX-156025098-156025397_WASH6P    540427     60591 -0.00858583  0.936434
chrX-156029831-156030380_WASH6P    540428     60591 -0.07580832  0.323499
chrX-156030497-156030869_WASH6P    540429     60591 -0.03870896  0.73

In [51]:
pairs.E2G.res = pairs.E2G[pairs.E2G$PairName %in% p2g$PairName]
mcols(pairs.E2G.res)[,c("Correlation","FDR","VarQATAC","VarQRNA")] = p2g[pairs.E2G.res$PairName,c("Correlation","FDR","VarQATAC","VarQRNA")]

In [52]:
pairs.E2G.res

GRanges object with 9357560 ranges and 7 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100004103-100004301      * |         AGL
        [2]     chr1 100004103-100004301      * |      CDC14A
        [3]     chr1 100004103-100004301      * |         DBT
        [4]     chr1 100004103-100004301      * |       EXTL2
        [5]     chr1 100004103-100004301      * |       FRRS1
        ...      ...                 ...    ... .         ...
  [9357556]     chrX   99980528-99980848      * |      PCDH19
  [9357557]     chrX   99980528-99980848      * |      TSPAN6
  [9357558]     chrX   99980528-99980848      * |        XKRX
  [9357559]     chrX     9999312-9999592      * |       TBL1X
  [9357560]     chrX     9999312-9999592      * |        WWC3
                          PeakName               PairName Correlation       FDR
                       <character>            <characte

Save results

In [53]:
saveRDS(obj.ArchR,
        paste(dir.output,"obj.ArchR.rds",sep = "/"))
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "Correlation",
                         "FDR",
                         "VarQATAC",
                         "VarQRNA")]
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
pairs.E2G.res
df.output

GRanges object with 9357560 ranges and 7 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100004103-100004301      * |         AGL
        [2]     chr1 100004103-100004301      * |      CDC14A
        [3]     chr1 100004103-100004301      * |         DBT
        [4]     chr1 100004103-100004301      * |       EXTL2
        [5]     chr1 100004103-100004301      * |       FRRS1
        ...      ...                 ...    ... .         ...
  [9357556]     chrX   99980528-99980848      * |      PCDH19
  [9357557]     chrX   99980528-99980848      * |      TSPAN6
  [9357558]     chrX   99980528-99980848      * |        XKRX
  [9357559]     chrX     9999312-9999592      * |       TBL1X
  [9357560]     chrX     9999312-9999592      * |        WWC3
                          PeakName               PairName Correlation       FDR
                       <character>            <characte

chr,start,end,TargetGene,CellType,Correlation,FDR,VarQATAC,VarQRNA
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,100004103,100004301,AGL,K562,-0.008354999,9.381026e-01,0.03058681,0.9207638
chr1,100004103,100004301,CDC14A,K562,-0.144103352,1.139215e-02,0.03058681,0.9925897
chr1,100004103,100004301,DBT,K562,0.035681633,7.579882e-01,0.03058681,0.8629334
chr1,100004103,100004301,EXTL2,K562,-0.055049899,5.549890e-01,0.03058681,0.8120513
chr1,100004103,100004301,FRRS1,K562,0.108421815,8.873393e-02,0.03058681,0.8764668
chr1,100004103,100004301,MFSD14A,K562,0.036598873,7.498066e-01,0.03058681,0.7454903
chr1,100004103,100004301,PLPPR5,K562,-0.034301369,7.699586e-01,0.03058681,0.7010942
chr1,100004103,100004301,RTCA,K562,0.065246670,4.375422e-01,0.03058681,0.8838606
chr1,100004103,100004301,RTCA-AS1,K562,0.124444601,3.836895e-02,0.03058681,0.7927415


In [54]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.9 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

Random number generation:
 RNG:     L'Ecuyer-CMRG 
 Normal:  Inversion 
 Sample:  Rejection 
 
locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] uwot_0.1.16            